In [1]:
import numpy as np

In [2]:
from abc import ABC, abstractmethod

class SupervisedMachineLearningModel(ABC):
    @abstractmethod
    def train(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass

### Linear Regression

#### Data
```
 ----------
|  X |  y  |
 ----------

y is continous value
```

#### 1. Model

```
y_pred = wX + b

where,
X = Independent variable
y_pred = Dependent variable
w = weight or slope
b = intercept or bias
```

#### 2. Loss

```
J(w) = (1 / (2 * m)) * Σ[(y_pred(i) - y(i))^2]

where,
m is the number of training examples.
y(i) is the actual output for the i-th example.
y_pred(i) is the predicted output for the i-th example.
```

#### 3. Gradient

```
∂J(w) / ∂wj = (1 / m) * Σ[(y_pred(i) - y(i)) * Xj(i)]

Compute the error term (y_pred(i) - y(i)).
Multiply the error by the corresponding feature Xj(i).
Take the average over all examples.
```

#### 4. Gradient Descent Update

```
wj = wj - α * ∂J(w) / ∂wj

where,
∂J(w) / ∂wj is gradient
```


In [3]:
class LinearRegression(SupervisedMachineLearningModel):
    def __init__(self, learning_rate: float = 0.01, epochs: int = 1000) -> None:
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None

    def train(self, X, y, verbose=False) -> None:
        """Train the linear regression model."""
        m, n = X.shape
        # Initialize weights (including bias as part of weights)
        self.weights = np.zeros(n + 1)

        # Add bias as the first column of X
        X = np.c_[np.ones(m), X]

        for epoch in range(self.epochs):
            # Predictions
            y_pred = X @ self.weights

            # Error calculation
            error = y_pred - y

            # Loss
            loss = (1 / (2 * m)) * np.sum(error ** 2)

            # Gradient calculation
            gradient = (1 / m) * (X.T @ error)

            # Update weights
            self.weights -= self.learning_rate * gradient

            if verbose:
                # Display
                print(f"iteration: ({epoch+1}/{self.epochs}) - error: {error} - loss: {loss} - gradient - {gradient}")

    def predict(self, X) -> np.ndarray:
        """Predict using the linear regression model."""
        m = X.shape[0]
        # Add bias as the first column of X
        X = np.c_[np.ones(m), X]
        return X @ self.weights

### Logistic Regression

#### Data

```
 ----------
|  X |  y  |
 ----------

y is discrete value
```

#### 1. Model

```
y_pred = sigmoid(X @ weights)

where,
sigmoid = 1/(1+exp(-z))

z = w0 + w1*x1 + w2*x2 + ... + wn*xn
w0 is bias term
```

#### 2. Loss

```
Cost = -(1/m) * Σ [y * log(y_pred) + (1 - y) * log(1 - y_pred)]
```

#### 3. Gradient Calculation

```
# Error
error = y_pred - y

# Gradient
gradient = (1 / m) * (X.T @ error)

# Weight update
weights -= learning_rate * gradient
```

In [22]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate: float = 0.01, epochs: int = 1000, threshold: float = 0.5):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.threshold = threshold
        self.weights = None
        self.mean = None
        self.std = None

    @staticmethod
    def __sigmoid(z):
        """Compute the sigmoid function."""
        return np.clip(1 / (1 + np.exp(-z)), 1e-10, 1 - 1e-10)  # Clipping for numerical stability

    def __standardize(self, X):
        """Standardize features natively (zero mean and unit variance)."""
        if self.mean is None or self.std is None:
            # Calculate mean and standard deviation for scaling
            self.mean = np.mean(X, axis=0)
            self.std = np.std(X, axis=0)
        return (X - self.mean) / self.std

    def train(self, X, y, verbose=False):
        """Train the logistic regression model."""
        m, n = X.shape

        # # Standardize the input features
        # X = self.__standardize(X)

        # Add bias term
        X = np.c_[np.ones(m), X]
        self.weights = np.zeros(n + 1)  # Initialize weights (including bias)

        for epoch in range(self.epochs):
            # Compute the linear combination
            z = X @ self.weights

            # Sigmoid activation
            y_pred = self.__sigmoid(z)

            # Compute the error
            error = y_pred - y

            # Compute the loss
            loss = -(1 / m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

            # Compute the gradient
            gradient = (1 / m) * (X.T @ error)

            # Update weights
            self.weights -= self.learning_rate * gradient

            if verbose:
                print(f"Iteration: ({epoch+1}/{self.epochs}) - Loss: {loss:.6f} - Gradient: {gradient}")

    def predict(self, X):
        """Make predictions using the logistic regression model."""
        # # Standardize the input features using the same mean and std as during training
        # X = self.__standardize(X)

        # Add bias term
        X = np.c_[np.ones(X.shape[0]), X]

        # Compute probabilities
        probabilities = self.__sigmoid(X @ self.weights)

        # print(f"probabilities: {probabilities}")

        # Apply threshold to get binary predictions
        return (probabilities >= self.threshold).astype(int)

### Test

In [4]:
! pip install ipytest --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00


In [24]:
import ipytest
ipytest.autoconfig()


def test_linear_regression():
    X = np.array([[1], [2], [3], [4], [5], [6], [7]])
    y = np.array([3, 5, 7, 9, 11, 13, 15])  # y = 2x + 1
    model = LinearRegression(learning_rate=0.01, epochs=1000)
    model.train(X[:4], y[:4], verbose=False)
    predictions = model.predict(X[5:])
    expected = y[5:]
    assert np.allclose(predictions, expected, rtol=1e-2, atol=1e-2, equal_nan=False), f"predictions {predictions} do not match expected {expected}"


def test_logistic_regression():
    X = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])
    y = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
    model = LogisticRegression(learning_rate=0.1, epochs=1000)
    model.train(X[:5], y[:5], verbose=False)
    predictions = model.predict(X[6:])
    expected = y[6:]
    assert np.allclose(predictions, expected, rtol=1e-2, atol=1e-2, equal_nan=False), f"predictions {predictions} do not match expected {expected}"


if __name__ == "__main__":
    ipytest.run()

.F                                                                                           [100%]
============================================= FAILURES =============================================
_____________________________________ test_logistic_regression _____________________________________

    def test_logistic_regression():
        X = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])
        y = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
        model = LogisticRegression(learning_rate=0.1, epochs=1000)
        model.train(X[:5], y[:5], verbose=False)
        predictions = model.predict(X[6:])
        expected = y[6:]
>       assert np.allclose(predictions, expected, rtol=1e-2, atol=1e-2, equal_nan=False), f"predictions {predictions} do not match expected {expected}"
E       AssertionError: predictions [1 1 1 1] do not match expected [1 0 1 0]
E       assert False
E        +  where False = <function allclose at 0x7f5a841eb330>(array([1, 1, 1, 1]), array([1, 0, 1